In [ ]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [ ]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn import tree
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import joblib

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [3]:
# Set features. This will also be used as your x values.
selected_features = df[[
    'koi_fpflag_nt',
    'koi_fpflag_ss',
    'koi_fpflag_co',
    'koi_fpflag_ec',
    'koi_period',
    'koi_time0bk',
    'koi_impact',
    'koi_duration',
    'koi_depth',
    'koi_prad',
    'koi_teq',
    'koi_insol',
    'koi_model_snr',
    'koi_tce_plnt_num',
    'koi_steff',
    'koi_slogg',
    'koi_srad',
    'ra',
    'dec',
    'koi_kepmag'
]]

selected_features.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,0,0,0,0,54.418383,162.513840,0.586,4.50700,874.8,2.83,443,9.11,25.8,2,5455,4.467,0.927,291.93423,48.141651,15.347
1,0,1,0,0,19.899140,175.850252,0.969,1.78220,10829.0,14.60,638,39.30,76.3,1,5853,4.544,0.868,297.00482,48.134129,15.436
2,0,1,0,0,1.736952,170.307565,1.276,2.40641,8079.2,33.46,1395,891.96,505.6,1,5805,4.564,0.791,285.53461,48.285210,15.597
3,0,0,0,0,2.525592,171.595550,0.701,1.65450,603.3,2.75,1406,926.16,40.9,1,6031,4.438,1.046,288.75488,48.226200,15.509
4,0,0,0,0,4.134435,172.979370,0.762,3.14020,686.0,2.77,1160,427.65,40.2,2,6046,4.486,0.972,296.28613,48.224670,15.714


# Create a Train Test Split

Use `koi_disposition` for the y values

In [4]:
target = df["koi_disposition"]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(selected_features, target, random_state=42)

In [6]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
6122,0,0,0,0,6.768901,133.077240,0.150,3.61600,123.1,1.24,1017,253.30,10.8,1,5737,4.327,1.125,294.40472,39.351681,14.725
6370,0,1,0,1,0.733726,132.020050,0.291,2.30900,114.6,0.86,1867,2891.64,13.8,1,5855,4.578,0.797,284.50391,42.463860,15.770
2879,1,0,0,0,7.652707,134.460380,0.970,79.89690,641.1,3.21,989,226.81,254.3,1,6328,4.481,0.963,295.50211,38.983540,13.099
107,0,0,0,0,7.953547,174.662240,0.300,2.63120,875.4,2.25,696,55.37,38.4,1,4768,4.536,0.779,291.15878,40.750271,15.660
29,0,0,0,0,4.959319,172.258529,0.831,2.22739,9802.0,12.21,1103,349.40,696.5,1,5712,4.359,1.082,292.16705,48.727589,15.263


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [7]:
# Scale your data

X_scaler = MinMaxScaler().fit(X_train)


In [8]:
# Transform the training and testing data using the X_scaler and y_scaler models

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Create the Decision Tree model



In [9]:

model1 = tree.DecisionTreeClassifier()
model1 =  model1.fit(X_train_scaled, y_train)


# Train the Decision Model



In [10]:
print(f"Training Data Score: {model1.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model1.score(X_test_scaled, y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.8518306636155606


In [11]:
sorted(zip(model1.feature_importances_, selected_features), reverse=True)

[(0.1949848042668497, 'koi_fpflag_nt'),
 (0.18322372199330472, 'koi_fpflag_co'),
 (0.17331823962024656, 'koi_fpflag_ss'),
 (0.147897189714782, 'koi_model_snr'),
 (0.037066439617984434, 'koi_impact'),
 (0.02983543937903135, 'koi_fpflag_ec'),
 (0.02767540421339612, 'ra'),
 (0.02357900546732984, 'koi_steff'),
 (0.020192180077646747, 'koi_period'),
 (0.020161214666582467, 'koi_duration'),
 (0.019566438794958924, 'koi_time0bk'),
 (0.0192851987170841, 'dec'),
 (0.01836418931327873, 'koi_prad'),
 (0.018220016648006616, 'koi_kepmag'),
 (0.014506712984304808, 'koi_srad'),
 (0.0130774732286925, 'koi_tce_plnt_num'),
 (0.01258180639295759, 'koi_teq'),
 (0.011746585104092929, 'koi_depth'),
 (0.008237779605578907, 'koi_insol'),
 (0.006480160193890686, 'koi_slogg')]

In [12]:
## select important features only

selected_features2 = df[[
    'koi_fpflag_nt',
    'koi_fpflag_ss',
    'koi_fpflag_co',
    'koi_fpflag_ec',
    'koi_impact',
    'koi_duration',
    'koi_model_snr',
    'koi_steff',
    'ra',
    'dec']]

selected_features2.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_impact,koi_duration,koi_model_snr,koi_steff,ra,dec
0,0,0,0,0,0.586,4.50700,25.8,5455,291.93423,48.141651
1,0,1,0,0,0.969,1.78220,76.3,5853,297.00482,48.134129
2,0,1,0,0,1.276,2.40641,505.6,5805,285.53461,48.285210
3,0,0,0,0,0.701,1.65450,40.9,6031,288.75488,48.226200
4,0,0,0,0,0.762,3.14020,40.2,6046,296.28613,48.224670


In [13]:
X_train, X_test, y_train, y_test = train_test_split(selected_features2, target, random_state=42)
# Scale your data

X_scaler = MinMaxScaler().fit(X_train)
# Transform the training and testing data using the X_scaler and y_scaler models

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:

model2 = tree.DecisionTreeClassifier()
model2 =  model2.fit(X_train_scaled, y_train)
print(f"Training Data Score: {model2.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model2.score(X_test_scaled, y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.8329519450800915


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [15]:
# param_grid={'min_samples_split': range(2, 403, 10)}
param_grid = { 'criterion':['gini','entropy'],'max_depth': np.arange(3, 15), 'min_samples_split': range(2, 403, 10)}
grid2 = GridSearchCV(model1, param_grid, cv = 7 , verbose=3)

In [16]:
# Train the model with GridSearch
grid2.fit(X_train_scaled, y_train)

Fitting 7 folds for each of 984 candidates, totalling 6888 fits
[CV] criterion=gini, max_depth=3, min_samples_split=2 ................
[CV]  criterion=gini, max_depth=3, min_samples_split=2, score=0.740, total=   0.0s
[CV] criterion=gini, max_depth=3, min_samples_split=2 ................
[CV]  criterion=gini, max_depth=3, min_samples_split=2, score=0.740, total=   0.0s
[CV] criterion=gini, max_depth=3, min_samples_split=2 ................
[CV]  criterion=gini, max_depth=3, min_samples_split=2, score=0.730, total=   0.0s
[CV] criterion=gini, max_depth=3, min_samples_split=2 ................
[CV]  criterion=gini, max_depth=3, min_samples_split=2, score=0.742, total=   0.0s
[CV] criterion=gini, max_depth=3, min_samples_split=2 ................
[CV]  criterion=gini, max_depth=3, min_samples_split=2, score=0.738, total=   0.0s
[CV] criterion=gini, max_depth=3, min_samples_split=2 ................
[CV]  criterion=gini, max_depth=3, min_samples_split=2, score=0.738, total=   0.0s
[CV] criteri

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s



[CV] criterion=gini, max_depth=3, min_samples_split=12 ...............
[CV]  criterion=gini, max_depth=3, min_samples_split=12, score=0.730, total=   0.0s
[CV] criterion=gini, max_depth=3, min_samples_split=12 ...............
[CV]  criterion=gini, max_depth=3, min_samples_split=12, score=0.742, total=   0.0s
[CV] criterion=gini, max_depth=3, min_samples_split=12 ...............
[CV]  criterion=gini, max_depth=3, min_samples_split=12, score=0.738, total=   0.0s
[CV] criterion=gini, max_depth=3, min_samples_split=12 ...............
[CV]  criterion=gini, max_depth=3, min_samples_split=12, score=0.738, total=   0.0s
[CV] criterion=gini, max_depth=3, min_samples_split=12 ...............
[CV]  criterion=gini, max_depth=3, min_samples_split=12, score=0.736, total=   0.0s
[CV] criterion=gini, max_depth=3, min_samples_split=22 ...............
[CV]  criterion=gini, max_depth=3, min_samples_split=22, score=0.740, total=   0.0s
[CV] criterion=gini, max_depth=3, min_samples_split=22 ..............

[CV]  criterion=gini, max_depth=3, min_samples_split=92, score=0.740, total=   0.0s
[CV] criterion=gini, max_depth=3, min_samples_split=92 ...............
[CV]  criterion=gini, max_depth=3, min_samples_split=92, score=0.740, total=   0.0s
[CV] criterion=gini, max_depth=3, min_samples_split=92 ...............
[CV]  criterion=gini, max_depth=3, min_samples_split=92, score=0.730, total=   0.0s
[CV] criterion=gini, max_depth=3, min_samples_split=92 ...............
[CV]  criterion=gini, max_depth=3, min_samples_split=92, score=0.742, total=   0.0s
[CV] criterion=gini, max_depth=3, min_samples_split=92 ...............
[CV]  criterion=gini, max_depth=3, min_samples_split=92, score=0.738, total=   0.0s
[CV] criterion=gini, max_depth=3, min_samples_split=92 ...............
[CV]  criterion=gini, max_depth=3, min_samples_split=92, score=0.738, total=   0.0s
[CV] criterion=gini, max_depth=3, min_samples_split=92 ...............
[CV]  criterion=gini, max_depth=3, min_samples_split=92, score=0.736, 

[CV]  criterion=gini, max_depth=3, min_samples_split=172, score=0.738, total=   0.0s
[CV] criterion=gini, max_depth=3, min_samples_split=172 ..............
[CV]  criterion=gini, max_depth=3, min_samples_split=172, score=0.738, total=   0.0s
[CV] criterion=gini, max_depth=3, min_samples_split=172 ..............
[CV]  criterion=gini, max_depth=3, min_samples_split=172, score=0.736, total=   0.0s
[CV] criterion=gini, max_depth=3, min_samples_split=182 ..............
[CV]  criterion=gini, max_depth=3, min_samples_split=182, score=0.740, total=   0.0s
[CV] criterion=gini, max_depth=3, min_samples_split=182 ..............
[CV]  criterion=gini, max_depth=3, min_samples_split=182, score=0.738, total=   0.0s
[CV] criterion=gini, max_depth=3, min_samples_split=182 ..............
[CV]  criterion=gini, max_depth=3, min_samples_split=182, score=0.730, total=   0.0s
[CV] criterion=gini, max_depth=3, min_samples_split=182 ..............
[CV]  criterion=gini, max_depth=3, min_samples_split=182, score=

...

[CV]  criterion=entropy, max_depth=14, min_samples_split=192, score=0.857, total=   0.1s
[CV] criterion=entropy, max_depth=14, min_samples_split=192 ..........
[CV]  criterion=entropy, max_depth=14, min_samples_split=192, score=0.881, total=   0.0s
[CV] criterion=entropy, max_depth=14, min_samples_split=192 ..........
[CV]  criterion=entropy, max_depth=14, min_samples_split=192, score=0.879, total=   0.0s
[CV] criterion=entropy, max_depth=14, min_samples_split=192 ..........
[CV]  criterion=entropy, max_depth=14, min_samples_split=192, score=0.852, total=   0.0s
[CV] criterion=entropy, max_depth=14, min_samples_split=192 ..........
[CV]  criterion=entropy, max_depth=14, min_samples_split=192, score=0.880, total=   0.0s
[CV] criterion=entropy, max_depth=14, min_samples_split=202 ..........
[CV]  criterion=entropy, max_depth=14, min_samples_split=202, score=0.883, total=   0.1s
[CV] criterion=entropy, max_depth=14, min_samples_split=202 ..........
[CV]  criterion=entropy, max_depth=14, m

[CV]  criterion=entropy, max_depth=14, min_samples_split=272, score=0.861, total=   0.0s
[CV] criterion=entropy, max_depth=14, min_samples_split=272 ..........
[CV]  criterion=entropy, max_depth=14, min_samples_split=272, score=0.888, total=   0.0s
[CV] criterion=entropy, max_depth=14, min_samples_split=272 ..........
[CV]  criterion=entropy, max_depth=14, min_samples_split=272, score=0.879, total=   0.0s
[CV] criterion=entropy, max_depth=14, min_samples_split=272 ..........
[CV]  criterion=entropy, max_depth=14, min_samples_split=272, score=0.846, total=   0.0s
[CV] criterion=entropy, max_depth=14, min_samples_split=272 ..........
[CV]  criterion=entropy, max_depth=14, min_samples_split=272, score=0.884, total=   0.0s
[CV] criterion=entropy, max_depth=14, min_samples_split=282 ..........
[CV]  criterion=entropy, max_depth=14, min_samples_split=282, score=0.885, total=   0.1s
[CV] criterion=entropy, max_depth=14, min_samples_split=282 ..........
[CV]  criterion=entropy, max_depth=14, m

[CV]  criterion=entropy, max_depth=14, min_samples_split=342, score=0.845, total=   0.1s
[CV] criterion=entropy, max_depth=14, min_samples_split=342 ..........
[CV]  criterion=entropy, max_depth=14, min_samples_split=342, score=0.881, total=   0.1s
[CV] criterion=entropy, max_depth=14, min_samples_split=352 ..........
[CV]  criterion=entropy, max_depth=14, min_samples_split=352, score=0.872, total=   0.1s
[CV] criterion=entropy, max_depth=14, min_samples_split=352 ..........
[CV]  criterion=entropy, max_depth=14, min_samples_split=352, score=0.912, total=   0.1s
[CV] criterion=entropy, max_depth=14, min_samples_split=352 ..........
[CV]  criterion=entropy, max_depth=14, min_samples_split=352, score=0.869, total=   0.0s
[CV] criterion=entropy, max_depth=14, min_samples_split=352 ..........
[CV]  criterion=entropy, max_depth=14, min_samples_split=352, score=0.869, total=   0.0s
[CV] criterion=entropy, max_depth=14, min_samples_split=352 ..........
[CV]  criterion=entropy, max_depth=14, m

[Parallel(n_jobs=1)]: Done 6888 out of 6888 | elapsed:  4.9min finished


GridSearchCV(cv=7, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                         'min_samples_split': range(2, 403, 10)},
             verbose=3)

In [17]:
print(grid2.best_params_)
print(grid2.best_score_)

{'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 122}
0.8817470913599085


# Save the Model

In [18]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
filename = 'DeepikaAwasthi_decisionTree.sav'
joblib.dump(grid2, filename)

['DeepikaAwasthi_decisionTree.sav']

In [19]:
# Make predictions with the hypertuned model
predictions = grid2.predict(X_test_scaled)

In [20]:
# Calculate classification report
print(classification_report(y_test, predictions,
                            target_names=["blue", "red", "yellow"]))

              precision    recall  f1-score   support

        blue       0.77      0.77      0.77       411
         red       0.81      0.80      0.80       484
      yellow       0.98      0.99      0.99       853

    accuracy                           0.89      1748
   macro avg       0.85      0.85      0.85      1748
weighted avg       0.88      0.89      0.88      1748



In [21]:
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

First 10 Predictions:   ['FALSE POSITIVE' 'CANDIDATE' 'FALSE POSITIVE' 'FALSE POSITIVE'
 'FALSE POSITIVE' 'CONFIRMED' 'CONFIRMED' 'CANDIDATE' 'CANDIDATE'
 'FALSE POSITIVE']
First 10 Actual labels: ['FALSE POSITIVE', 'CANDIDATE', 'FALSE POSITIVE', 'FALSE POSITIVE', 'FALSE POSITIVE', 'CONFIRMED', 'CANDIDATE', 'CANDIDATE', 'CANDIDATE', 'FALSE POSITIVE']


In [22]:
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)

,Prediction,Actual
0,FALSE POSITIVE,FALSE POSITIVE
1,CANDIDATE,CANDIDATE
2,FALSE POSITIVE,FALSE POSITIVE
3,FALSE POSITIVE,FALSE POSITIVE
4,FALSE POSITIVE,FALSE POSITIVE
...,...,...
1743,CANDIDATE,CANDIDATE
1744,CANDIDATE,CANDIDATE
1745,CONFIRMED,CONFIRMED
1746,CONFIRMED,CONFIRMED
